In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext(master='local', appName='transformaciones_acciones')

23/04/03 06:16:19 WARN Utils: Your hostname, PRRS-DCARVAJAL resolves to a loopback address: 127.0.1.1; using 172.17.19.227 instead (on interface eth0)
23/04/03 06:16:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/03 06:16:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
rdd1 = sc.parallelize([1,2,3])
type(rdd1)

pyspark.rdd.RDD

In [4]:
rdd1.collect()

[1, 2, 3]

In [5]:
sc

<SparkContext master=local appName=transformaciones_acciones>

In [6]:
### Load data
path='./files/'

In [7]:
equipos_olimpicos_rdd = sc.textFile(path+'paises.csv').map(lambda line:line.split(','))

In [8]:
equipos_olimpicos_rdd.take(5)

[['id', 'equipo', 'sigla'],
 ['1', '30. Februar', 'AUT'],
 ['2', 'A North American Team', 'MEX'],
 ['3', 'Acipactli', 'MEX'],
 ['4', 'Acturus', 'ARG']]

#### Class 09

In [9]:
equipos_olimpicos_rdd.map(lambda x:(x[2])).distinct().count()

231

In [10]:
equipos_olimpicos_rdd.map(lambda x:(x[2], x[1])).groupByKey().mapValues(len).take(5)

[('sigla', 1), ('AUT', 11), ('MEX', 9), ('ARG', 18), ('AFG', 1)]

In [11]:
equipos_olimpicos_rdd.map(lambda x:(x[2], x[1])).groupByKey().mapValues(list).take(2)

[('sigla', ['equipo']),
 ('AUT',
  ['30. Februar',
   'Austria',
   'Austria-1',
   'Austria-2',
   'Breslau',
   'Brigantia',
   'Donar III',
   'Evita VI',
   'May-Be 1960',
   '"R.-V. Germania; Leitmeritz"',
   'Surprise'])]

In [12]:
equipos_ARG = equipos_olimpicos_rdd.filter(lambda x: 'ARG' in x)

In [13]:
equipos_ARG.collect()

[['4', 'Acturus', 'ARG'],
 ['37', 'Antares', 'ARG'],
 ['42', 'Arcturus', 'ARG'],
 ['43', 'Ardilla', 'ARG'],
 ['45', 'Argentina', 'ARG'],
 ['46', 'Argentina-1', 'ARG'],
 ['47', 'Argentina-2', 'ARG'],
 ['119', 'Blue Red', 'ARG'],
 ['238', 'Covunco III', 'ARG'],
 ['252', 'Cupidon III', 'ARG'],
 ['288', 'Djinn', 'ARG'],
 ['436', 'Gullvinge', 'ARG'],
 ['644', 'Matrero II', 'ARG'],
 ['672', 'Mizar', 'ARG'],
 ['774', 'Pampero', 'ARG'],
 ['843', 'Rampage', 'ARG'],
 ['1031', 'Tango', 'ARG'],
 ['1162', 'Wiking', 'ARG']]

In [14]:
equipos_olimpicos_rdd.count()

1185

In [15]:
# contar hasta que se acaben los 20 milisegundos.
equipos_olimpicos_rdd.countApprox(20)

1185

### Class 10

In [16]:
deportista_olimpico_rdd = sc.textFile(path+'deportista.csv').map(lambda l:l.split(','))
deportista_olimpico_rdd2 = sc.textFile(path+'deportista2.csv').map(lambda l:l.split(','))
deportista_olimpico_rdd = deportista_olimpico_rdd.union(deportista_olimpico_rdd2)

In [17]:
deportista_olimpico_rdd.count()

135572

In [18]:
equipos_olimpicos_rdd.top(5)

[['id', 'equipo', 'sigla'],
 ['999', 'Stella-2', 'NOR'],
 ['998', 'State VI', 'CAN'],
 ['997', 'Starlight III', 'GBR'],
 ['996', 'Starita', 'NED']]

In [19]:
deportista_olimpico_rdd.top(5)

[['deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo_id'],
 ['99999', 'Alexander Grant Alick Rennie', '1', '32', '182', '71', '967'],
 ['99998', 'Robert John Bob Renney', '1', '21', '178', '90', '66'],
 ['99997', 'Thomas Renner', '1', '24', '183', '86', '71'],
 ['99996', 'Sara Renner', '2', '21', '168', '63', '174']]

In [20]:
deportista_olimpico_rdd.map(lambda l:(l[-1],l[:-1])).join(equipos_olimpicos_rdd.map(lambda x:(x[0],x[2]))).take(10)

[('199', (['1', 'A Dijiang', '1', '24', '180', '80'], 'CHN')),
 ('199', (['2', 'A Lamusi', '1', '23', '170', '60'], 'CHN')),
 ('199', (['602', 'Abudoureheman', '1', '22', '182', '75'], 'CHN')),
 ('199', (['1463', 'Ai Linuer', '1', '25', '160', '62'], 'CHN')),
 ('199', (['1464', 'Ai Yanhan', '2', '14', '168', '54'], 'CHN')),
 ('199', (['3605', 'An Weijiang', '1', '22', '178', '72'], 'CHN')),
 ('199', (['3610', 'An Yulong', '1', '19', '173', '70'], 'CHN')),
 ('199', (['3611', 'An Zhongxin', '2', '23', '170', '65'], 'CHN')),
 ('199', (['4639', 'Ao Changrong', '1', '25', '173', '71'], 'CHN')),
 ('199', (['4641', 'Ao Tegen', '1', '21', '181', '90'], 'CHN'))]

In [21]:
# take sample randomly base in a seed
deportista_olimpico_rdd.map(lambda l:(l[-1],l[:-1])).join(equipos_olimpicos_rdd.map(lambda x:(x[0],x[2]))).takeSample(False, 10, 333)

[('656',
  (['105216', 'Mara Guadalupe Snchez Gmez', '2', '29', '169', '53'], 'MEX')),
 ('93',
  (['21394', 'Iosif Ivanovich Chugoshvili', '1', '25', '188', '120'], 'BLR')),
 ('174', (['120246', 'Paul Tichelaar', '1', '25', '184', '73'], 'CAN')),
 ('1121', (['70484', 'Hildemaro P. Lista', '1', '31', '0', '0'], 'URU')),
 ('203', (['116588', 'Sung YiChieh', '2', '18', '164', '57'], 'TPE')),
 ('507', (['15862', 'Antonio Brutti', '1', '27', '169', '59'], 'ITA')),
 ('362', (['47754', 'Ludovic Henry', '1', '47', '195', '88'], 'FRA')),
 ('96', (['13326', 'Jill Boon', '2', '25', '172', '65'], 'BEL')),
 ('810', (['87205', 'Marcin Nowak', '1', '20', '215', '105'], 'POL')),
 ('1026', (['110006', 'Zouheir AlShourbagi', '1', '0', '0', '0'], 'SYR'))]

In [22]:
resultados = sc.textFile(path+'resultados.csv').map(lambda l:l.split(','))
resultados.count()

271117

In [23]:
resultados.top(5)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['99999', 'NA', '50604', '51', '568'],
 ['99998', 'NA', '50603', '47', '36'],
 ['99997', 'NA', '50602', '49', '262'],
 ['99996', 'NA', '50601', '47', '614']]

In [24]:
ganadores = resultados.filter(lambda l:'NA' not in l[1])
ganadores.top(5)

[['resultado_id', 'medalla', 'deportista_id', 'juego_id', 'evento_id'],
 ['99993', 'Silver', '50600', '39', '207'],
 ['99986', 'Silver', '50597', '45', '199'],
 ['99985', 'Bronze', '50597', '43', '72'],
 ['99963', 'Bronze', '50580', '37', '245']]

### Class 11

In [28]:
deportista_paises = deportista_olimpico_rdd.map(lambda l:[l[-1],l[:-1]]).join(equipos_olimpicos_rdd.map(lambda x:[x[0],x[2]]))
deportista_paises.top(5)

[('999', (['92679', 'Trygve Bjarne Pedersen', '1', '35', '0', '0'], 'NOR')),
 ('999', (['1144', 'Henrik Agersborg', '1', '47', '0', '0'], 'NOR')),
 ('999', (['10765', 'Einar Berntsen', '1', '28', '0', '0'], 'NOR')),
 ('998',
  (['111659', 'G. Bernard Bernie Skinner', '1', '34', '182', '82'], 'CAN')),
 ('996', (['116030', 'Edward Eddy Stutterheim', '1', '39', '0', '0'], 'NED'))]

In [30]:
medallas = deportista_paises.join(ganadores)
medallas.take(6)

[('74',
  ((['65', 'Patimat Abakarova', '2', '21', '165', '49'], 'AZE'), 'Gold')),
 ('74', ((['129', 'Ruslan Abbasov', '1', '22', '181', '74'], 'AZE'), 'Gold')),
 ('74', ((['130', 'Tural Abbasov', '1', '18', '182', '76'], 'AZE'), 'Gold')),
 ('74', ((['131', 'Tran Abbasova', '2', '33', '159', '53'], 'AZE'), 'Gold')),
 ('74',
  ((['335', 'Abdulqdir Abdullayev', '1', '28', '188', '91'], 'AZE'), 'Gold')),
 ('74',
  ((['336', 'Arif Yadulla Abdullayev', '1', '27', '164', '60'], 'AZE'),
   'Gold'))]

### Class 12

In [36]:
from operator import add

In [34]:
points = {'Gold':7,'Silver':5,'Bronze':4}

In [37]:
country_point = medallas.map(lambda x: (x[1][0][-1], points[x[1][1]]))

In [40]:
results2 = country_point.reduceByKey((add)).sortBy(lambda x:x[1], ascending=False)

In [41]:
results2.take(10)

[('CAN', 32538),
 ('ARG', 12520),
 ('HUN', 10860),
 ('MEX', 6124),
 ('RSA', 3788),
 ('BLR', 3580),
 ('LTU', 1535),
 ('MGL', 1460),
 ('USA', 1342),
 ('AZE', 1218)]

In [45]:
sc.stop()